# Training and hosting SageMaker Models using the Apache MXNet Module API

The **SageMaker Python SDK** makes it easy to train and deploy MXNet models. In this example, we train a simple neural network using the Apache MXNet [Module API](https://mxnet.apache.org/api/python/module/module.html) and the driving score dataset.

### Setup

First we need to define a few variables that will be needed later in the example.

In [1]:
!pip install sagemaker==1.38.6
!pip freeze

    100% |████████████████████████████████| 225kB 20.2MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 27.9MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 26.5MB/s ta 0:00:01
    100% |████████████████████████████████| 5.7MB 4.9MB/s eta 0:00:01
    100% |████████████████████████████████| 215kB 39.7MB/s ta 0:00:01


  Running setup.py bdist_wheel for sagemaker ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/75/cc/82/52125309cf9f3667c544ed09bc875c94f865e91974e7de4110
Successfully built sagemaker
awscli 1.16.198 has requirement botocore==1.12.188, but you'll have botocore 1.12.225 which is incompatible.
  Found existing installation: botocore 1.12.188
    Uninstalling botocore-1.12.188:
      Successfully uninstalled botocore-1.12.188
  Found existing installation: boto3 1.9.188
    Uninstalling boto3-1.9.188:
      Successfully uninstalled boto3-1.9.188
  Found existing installation: sagemaker 1.33.0
    Uninstalling sagemaker-1.33.0:
      Successfully uninstalled sagemaker-1.33.0
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
alabaster==0.7.10
anaconda-client==1.6.14
anaconda-project==0.8.2
asn1crypto==0.24.0
astroid==1.6.3
astropy==2.0.6
attrs==18.1.0
autovizwidget==0.12.7
awscli==

In [2]:
from sagemaker import get_execution_role
from sagemaker.session import Session

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket here if you wish.
bucket = Session().default_bucket()

# Location to save your custom code in tar.gz format.
custom_code_upload_location = 's3://{}/customcode/mxnet'.format(bucket)

# Location where results of model training are saved.
model_artifacts_location = 's3://{}/artifacts'.format(bucket)

In [3]:
# IAM execution role that gives SageMaker access to resources in your AWS account.
# We can use the SageMaker Python SDK to get the role from our notebook environment. 
role = get_execution_role()

### The training script

The ``maintenance.py`` script provides all the code we need for training and hosting a SageMaker model. The script also checkpoints the model at the end of every epoch and saves the model graph, params and optimizer state in the folder `/opt/ml/checkpoints`. If the folder path does not exist then it will skip checkpointing.

In [4]:
print(bucket)

sagemaker-us-east-1-540323643652


In [5]:
!cat train.py

import argparse
import gzip
import json
import logging
import os
import struct
import boto3

import mxnet as mx
import numpy as np

import pickle
# from sklearn.preprocessing import StandardScaler
# from sklearn.utils import shuffle


def load_pickle_file(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)

    return data

def load_data():
    data = load_pickle_file('/opt/ml/code/data.pickle')
    label = load_pickle_file('/opt/ml/code/label.pickle')
    
    return split_data(data, label)

def split_data(data, label):

    
    X, y = (data, label)
    # split dataset
    train_data = X[:80, :].astype('float32')

    train_label = y[:80]
    val_data = X[80 :].astype('float32')
    val_label = y[80:]
    return train_data, train_label, val_data, val_label

def build_graph():
    data = mx.sym.var('data')
    fc1 = mx.sym.FullyConnected(data=data, name='fc1', num_hidden=128)
    act1 = mx.sym.Activation(data=fc1, name='act1', act_type="relu")
    fc2 = mx.sym.FullyCo

### SageMaker's MXNet estimator class

The SageMaker ```MXNet``` estimator allows us to run single machine or distributed training in SageMaker, using CPU or GPU-based instances.

When we create the estimator, we pass in the filename of our training script, the name of our IAM execution role, and the S3 locations we defined in the setup section. We also provide a few other parameters. ``train_instance_count`` and ``train_instance_type`` determine the number and type of SageMaker instances that will be used for the training job. The ``hyperparameters`` parameter is a ``dict`` of values that will be passed to your training script -- you can see how to access these values in the ``train.py`` script above.

For this example, we will choose one ``ml.m4.xlarge`` instance.

In [6]:
from sagemaker.mxnet import MXNet

drvscore_estimator = MXNet(entry_point='train.py',
                           role=role,
                           output_path=model_artifacts_location,
                           code_location=custom_code_upload_location,
                           train_instance_count=1,
                           train_instance_type='ml.m4.xlarge',
                           framework_version='1.2.1',
                           py_version='py2',
                           dependencies=['data/data.pickle','data/label.pickle'])

No handlers could be found for logger "sagemaker"


### Running the Training Job

After we've constructed our MXNet object, we can fit it using data stored in S3. Below we run SageMaker training on two input channels: **train** and **test**.

During training, SageMaker makes this data stored in S3 available in the local filesystem where the script is running. The ```train.py``` script simply loads the train and test data from disk.

In [7]:
%%time
import boto3

region = boto3.Session().region_name

CPU times: user 16 ms, sys: 13.7 ms, total: 29.7 ms
Wall time: 24.6 ms


In [8]:
drvscore_estimator.fit(job_name='connectedcar-drvscore8')

2019-09-10 17:47:16 Starting - Starting the training job..................
2019-09-10 17:50:14 Starting - Launching requested ML instances.........
2019-09-10 17:51:19 Starting - Preparing the instances for training......
2019-09-10 17:52:44 Downloading - Downloading input data
2019-09-10 17:52:44 Training - Downloading the training image...
2019-09-10 17:53:14 Uploading - Uploading generated training model
2019-09-10 17:53:03,290 INFO - root - running container entrypoint
2019-09-10 17:53:03,290 INFO - root - starting train task
2019-09-10 17:53:03,297 INFO - container_support.training - Training starting
2019-09-10 17:53:04,188 WARNING - mxnet_container.train - #033This required structure for training scripts will be deprecated with the next major release of MXNet images. The train() function will no longer be required; instead the training script must be able to be run as a standalone script. For more information, see https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagem

2019-09-10 17:53:11,687 INFO - root - Epoch[30] Train-accuracy=0.225000
2019-09-10 17:53:11,688 INFO - root - Epoch[30] Time cost=0.010
2019-09-10 17:53:11,693 INFO - root - Epoch[30] Validation-accuracy=0.100000
2019-09-10 17:53:11,704 INFO - root - Epoch[31] Train-accuracy=0.225000
2019-09-10 17:53:11,704 INFO - root - Epoch[31] Time cost=0.011
2019-09-10 17:53:11,710 INFO - root - Epoch[31] Validation-accuracy=0.100000
2019-09-10 17:53:11,720 INFO - root - Epoch[32] Train-accuracy=0.250000
2019-09-10 17:53:11,720 INFO - root - Epoch[32] Time cost=0.010
2019-09-10 17:53:11,724 INFO - root - Epoch[32] Validation-accuracy=0.100000
2019-09-10 17:53:11,731 INFO - root - Epoch[33] Train-accuracy=0.250000
2019-09-10 17:53:11,731 INFO - root - Epoch[33] Time cost=0.007
2019-09-10 17:53:11,735 INFO - root - Epoch[33] Validation-accuracy=0.100000
2019-09-10 17:53:11,742 INFO - root - Epoch[34] Train-accuracy=0.237500
2019-09-10 17:53:11,742 INFO - root - Epoch[34] Time cost=0.007
2019-09-10 1

# (Optional) Creating an inference Endpoint

After training, we use the ``MXNet estimator`` object to build and deploy an ``MXNetPredictor``. This creates a Sagemaker **Endpoint** -- a hosted prediction service that we can use to perform inference. 

The arguments to the ``deploy`` function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances. Here we will deploy the model to a single ``ml.m4.xlarge`` instance.

In [9]:
#predictor = drvscore_estimator.deploy(initial_instance_count=1,
#                                   instance_type='ml.m4.xlarge')

# (Optional) Making an inference request

Now that our Endpoint is deployed and we have a ``predictor`` object, we can use it to run inference.

In [10]:
# Load data you want to run inference
#data = ...
#response = predictor.predict(data)

# (Optional) Delete the Endpoint

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it.

In [11]:
#print("Endpoint name: " + predictor.endpoint)
#predictor.delete_endpoint()